In [1]:
from time import sleep
from lxml.etree import HTML
from selenium import webdriver
import pandas as pd

In [8]:
def get_band_urls(number_bands=730):
    base_url = "http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Search/FNMain.aspx?BAND_NUMBER="
    end_url = "&lang=eng"
    urls = []
    for i in range(1,number_bands+1):
        urls.append(base_url + str(i) + end_url)   
        
    return urls

def filter_bands(bands, key, whitelist):
    filteredBands = bands[bands[key].isin(whitelist)]
    return filteredBands

def extract_band_information_links(url):
    links = {}
    links['reserves'] = url.replace('FNMain','FNReserves')
    links['governance'] = url.replace('FNMain','FNGovernance')
    links['federal_funding'] = url.replace('FNMain','FederalFundsMain')
    links['geography'] = url.replace('FNMain','FNGeography')
    links['registered_pop'] = url.replace('FNMain','FNRegPopulation')
    links['fnfta'] = url.replace('FNMain','FederalFundingMain')
    return links

def extract_band_census_links(url):
    links = {}
    links['population'] = url.replace('FNMain','FNPopulation')
    links['language'] = url.replace('FNMain','FNLanguage')
    links['marital'] = url.replace('FNMain','FNMartial')
    links['education'] = url.replace('FNMain','FNEducation')
    links['income'] = url.replace('FNMain','FNIncome')
    links['workforce'] = url.replace('FNMain','FNWorkforce')
    links['Mobility'] = url.replace('FNMain','FNMobility')
    links['Household'] = url.replace('FNMain','FNHousehold')
    return links


def read_allBands():
    allBands = pd.read_csv('allBands.csv')
    allBands['bandNumber'] = allBands.index + 1
    allBands['Province'] = allBands['address'].apply(lambda x: x.split(' ')[-1])
    allBands['url'] = get_band_urls()
    allBands = allBands.drop_duplicates(subset ="bandName", keep = 'first') 
    allBands['information_links'] = allBands['url'].apply(extract_band_information_links)
    allBands['census_links'] = allBands['url'].apply(extract_band_census_links)
    return allBands


Get all bands and their information links.

In [10]:
allBands = read_allBands()

Filter bands.

In [11]:
key ='bandName'
whitelist = ['Lennox Island']
filtered_bands = filter_bands(allBands, key, whitelist)

Get filtered bands links.

In [12]:
governance_links = filtered_bands['information_links'].apply(lambda x: x['governance'])
federal_funding_links = filtered_bands['information_links'].apply(lambda x: x['federal_funding'])
geography_links = filtered_bands['information_links'].apply(lambda x: x['geography'])
registered_pop_links = filtered_bands['information_links'].apply(lambda x: x['registered_pop'])

Reserves Data

In [13]:
def get_reserve(browser, base_url, relative_url):
    reserve_link = base_url.split('FNReserves')[0] + relative_url
    browser.get(reserve_link)
    html = browser.page_source
    table = pd.read_html(html, attrs = {'id': 'plcMain_dgFNlist'})[0]
    table.columns = ['bandNumber','bandName','address']
    return table

def get_band_reserves_table(url, browser):
    
    browser.get(url)
    html = browser.page_source
    table_1 = pd.read_html(html, attrs = {'id': 'plcMain_dgReservesList'})[0]

    tree = HTML(html)
    links = [l.values()[2] for l in tree.xpath("//a[starts-with(@id,'plcMain_dgReservesList_hlRVName')]")]
    table_2 = pd.concat([get_reserve(browser, url, relative_url) for relative_url in links])

    reserves_table = table_1.merge(table_2.reset_index(drop=True), left_index=True,right_index=True)
    
    return reserves_table

def get_bands_reserves_data(bands, browser):
    reserves_links = bands['information_links'].apply(lambda x: x['reserves'])
    reserves_tables = []
    for reserve_url in reserves_links.values:
        try:
            reserves_table = get_band_reserves_table(reserve_url, browser)
            reserves_tables.append(reserves_table)
        except Exception as e:
            print("Failed on {}".format(reserve_url))
            print(e)
        
    return pd.concat(reserves_tables)

In [194]:
browser = webdriver.Firefox()

In [208]:
reserves_data = get_bands_reserves_data(allBands, browser)

Failed on http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Search/FNReserves.aspx?BAND_NUMBER=34&lang=eng
No tables found
Failed on http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Search/FNReserves.aspx?BAND_NUMBER=53&lang=eng
No tables found
Failed on http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Search/FNReserves.aspx?BAND_NUMBER=165&lang=eng
No tables found
Failed on http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Search/FNReserves.aspx?BAND_NUMBER=326&lang=eng
No tables found
Failed on http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Search/FNReserves.aspx?BAND_NUMBER=492&lang=eng
No tables found
Failed on http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Search/FNReserves.aspx?BAND_NUMBER=496&lang=eng
No tables found
Failed on http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Search/FNReserves.aspx?BAND_NUMBER=497&lang=eng
No tables found
Failed on http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Search/FNReserves.aspx?BAND_NUMBER=503&lang=eng
No tables found
Failed on http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Search/FNReserves.aspx

In [210]:
reserves_data.reset_index(drop=True)

,No.,Name,Location,Hectares,bandNumber,bandName,address
0,6000,MORELL 2,38.4 KM NE/NE OF/DE CHARLOTTETOWN,83.0,1,Abegweit,"PO BOX 36, 11222 ST. PETERS ROAD, SCOTCHFORT, ..."
1,6001,ROCKY POINT 3,S.S. OF CHARLOTTETOWN HARBOUR,4.9,1,Abegweit,"PO BOX 36, 11222 ST. PETERS ROAD, SCOTCHFORT, ..."
2,6002,SCOTCHFORT 4,24 KM NE/NE OF/DE CHARLOTTETOWN,113.1,1,Abegweit,"PO BOX 36, 11222 ST. PETERS ROAD, SCOTCHFORT, ..."
3,6003,LENNOX ISLAND 1,24 KM N/N OF/DE SUMMERSIDE,535.1,2,Lennox Island,"PO BOX 134, LENNOX ISLAND, PE, C0B 1P0"
4,9942,LENNOX ISLAND NO.6,NaN,9.7,2,Lennox Island,"PO BOX 134, LENNOX ISLAND, PE, C0B 1P0"
...,...,...,...,...,...,...,...
2887,8489,CHUNDOO LH'TAN LA 45,LOT 3023 RANGE 5 COAST DISTRICT,42.9,730,Binche Whut'en,"204 Mandine Drive, PO Box 2039, Fort St. James..."
2888,7562,CHUZ GHUN 8,"COAST DISTRICT, RANGE 5, LOT 2165, ON TRAIL MI...",16.2,730,Binche Whut'en,"204 Mandine Drive, PO Box 2039, Fort St. James..."
2889,8473,LHOH CHO 29,LOT 1961 RANGE 5 COAST DISTRICT,0.4,730,Binche Whut'en,"204 Mandine Drive, PO Box 2039, Fort St. James..."
2890,8470,O K'AY WHA CHO 26,BLOCK A LOT 4686 RANGE 5 COAST DISTRICT,0.4,730,Binche Whut'en,"204 Mandine Drive, PO Box 2039, Fort St. James..."


In [211]:
reserves_data.reset_index(drop=True).to_csv("allReserves.csv",index=False)

In [219]:
allBands

,bandName,bandNumber,address,postalCode,txtPone,website,url,Province,information_links
0,Abegweit,1,"PO BOX 36, 11222 ST. PETERS ROAD, SCOTCHFORT, PE",C0A 1T0,(902) 676-2353,http://www.abegweitfirstnations.com/,http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Sear...,PE,{'reserves': 'http://fnp-ppn.aadnc-aandc.gc.ca...
1,Lennox Island,2,"PO BOX 134, LENNOX ISLAND, PE",C0B 1P0,(902) 831-2779,http://www.lennoxisland.com/,http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Sear...,PE,{'reserves': 'http://fnp-ppn.aadnc-aandc.gc.ca...
2,Elsipogtog First Nation,3,", 373 BIG COVE ROAD, ELSIPOGTOG FIRST NATION, NB",E4W 2S3,(506) 523-8200,http://www.elsipogtog.ca/,http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Sear...,NB,{'reserves': 'http://fnp-ppn.aadnc-aandc.gc.ca...
3,Buctouche MicMac,4,"9 RESERVE ROAD, BUCTOUCHE RESERVE, NB",E4S 4G2,(506) 743-2520,http://www.nsmdc.ca/main.html,http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Sear...,NB,{'reserves': 'http://fnp-ppn.aadnc-aandc.gc.ca...
4,Esgenoopetitj First Nation,5,"621 BAYVIEW DRIVE, BURNT CHURCH, NB",E9G 2A8,(506) 776-1200,http://esgenoopetitjfirstnation.org/,http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Sear...,NB,{'reserves': 'http://fnp-ppn.aadnc-aandc.gc.ca...
...,...,...,...,...,...,...,...,...,...
720,Lhoosk'uz Dene Nation,721,"PO BOX 4339, QUESNEL, BC",V2J 3J8,(250) 992-3290,http://www.lhooskuz.com/,http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Sear...,BC,{'reserves': 'http://fnp-ppn.aadnc-aandc.gc.ca...
722,Stswecem'c Xgat'tem First Nation,723,"GENERAL DELIVERY, DOG CREEK, BC",V0L 1J0,(250) 440-5645,http://canoecreekband.ca/,http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Sear...,BC,{'reserves': 'http://fnp-ppn.aadnc-aandc.gc.ca...
724,Wet'suwet'en First Nation,725,"PO BOX 760, BURNS LAKE, BC",V0J 1E0,(250) 698-7307,http://www.cstc.bc.ca/cstc/44/wetsuweten+first...,http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Sear...,BC,{'reserves': 'http://fnp-ppn.aadnc-aandc.gc.ca...
727,Yekooche First Nation,728,"1890 - 3RD AVENUE, PRINCE GEORGE, BC",V2M 1G4,(250) 562-0592,http://www.yekooche.com/,http://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Sear...,BC,{'reserves': 'http://fnp-ppn.aadnc-aandc.gc.ca...


In [222]:
band_province_hectares = allBands.merge(reserves_data.groupby('bandName').sum()[['Hectares']], left_on='bandName',right_index=True)[['bandName','Province','Hectares']]

In [246]:
band_province_hectares.to_csv('bandsProvinceHectares.csv',index=False)

In [232]:
stats = band_province_hectares.groupby('Province').count()

In [234]:
stats['Hectares(Sum)'] = band_province_hectares.groupby('Province')['Hectares'].sum()

In [239]:
stats = stats.drop('Hectares',axis=1).rename({'bandName':'Bands(count)'},axis=1)

In [243]:
stats['Hectares/Band(Mean)'] = stats['Hectares(Sum)'] / stats['Bands(count)']

In [249]:
stats.to_csv('bandProvinceHectareStats.csv')

In [248]:
stats

,Bands(count),Hectares(Sum),Hectares/Band(Mean)
Province,,,
AB,27,588804.4,21807.570370
BC,139,304147.5,2188.111511
MB,39,285048.0,7308.923077
NB,13,15592.2,1199.400000
NL,1,2839.0,2839.000000
NS,8,9031.2,1128.900000
NT,1,10049.7,10049.700000
ON,100,709263.5,7092.635000
PE,2,764.6,382.300000


In [213]:
reserves_data.sum()['Hectares']

3214456.4000000004

In [165]:
df

,bandNumber,bandName,address
0,2,Lennox Island,"PO BOX 134, LENNOX ISLAND, PE, C0B 1P0"
0,2,Lennox Island,"PO BOX 134, LENNOX ISLAND, PE, C0B 1P0"
0,2,Lennox Island,"PO BOX 134, LENNOX ISLAND, PE, C0B 1P0"
